Question 1 - Extracting Tesla Stock Data Using yfinance

In [4]:
import yfinance as yf
ticker_symbol = "TSLA"
tesla_stock = yf.Ticker(ticker_symbol)
tesla_data = tesla_stock.history(period="max")
tesla_data.reset_index(inplace=True)
print(tesla_data.head())

                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://finance.yahoo.com/quote/TSLA/history/"
data  =  requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')
read_html_pandas_data = pd.read_html(url)
read_html_pandas_data = pd.read_html(str(soup))
tesla_dataframe = read_html_pandas_data[0]

tesla_dataframe.head()

HTTPError: HTTP Error 404: Not Found

Question 3 - Extracting GameStop Stock Data Using yfinance

In [29]:
ticker_symbol = "GME"
gme_data = yf.download(ticker_symbol, start="2020-01-01", end="2024-01-01", progress=False)
print(gme_data.head())

Price                     Adj Close   Close    High     Low    Open    Volume
Ticker                          GME     GME     GME     GME     GME       GME
Date                                                                         
2020-01-02 00:00:00+00:00    1.5775  1.5775  1.6175  1.5175  1.5350  17814400
2020-01-03 00:00:00+00:00    1.4700  1.4700  1.5625  1.4600  1.5525  14175600
2020-01-06 00:00:00+00:00    1.4625  1.4625  1.4775  1.4000  1.4500  13579200
2020-01-07 00:00:00+00:00    1.3800  1.3800  1.4575  1.3600  1.4425  20912000
2020-01-08 00:00:00+00:00    1.4300  1.4300  1.4625  1.3525  1.3725  22517600


Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [32]:
url = "https://finance.yahoo.com/quote/GME/financials?p=GME"
response = requests.get(url)
if response.status_code == 200:
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table that contains the income statement data
    table = soup.find('div', {'class': 'Mb(25px)'})

    # Check if table is found
    if table:
        # Find all rows within the table
        rows = table.find_all('div', {'class': 'D(tbr)'})

        # Extract revenue data (usually the first or second row in the table)
        revenue_data = {}

        for row in rows:
            # Find the column cells in each row
            cols = row.find_all('div', {'class': 'D(ib)'})
            if len(cols) > 1:
                # Get the period and revenue data
                period = cols[0].text.strip()
                revenue = cols[1].text.strip()

                # Store the revenue data in the dictionary
                revenue_data[period] = revenue

        # Print the extracted revenue data
        print("GameStop Revenue Data (by Period):")
        for period, revenue in revenue_data.items():
            print(f"{period}: {revenue}")
    else:
        print("Could not find the income statement table.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Failed to retrieve the page. Status code: 404


Question 5 - Tesla Stock and Revenue Dashboard

In [37]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch Tesla Stock Data
ticker = "TSLA"
stock_data = yf.download(ticker, start="2020-01-01", end="2024-01-01", progress=False)

# Step 2: Fetch Tesla Revenue Data from Yahoo Finance (using Web Scraping)
def get_tesla_revenue():
    url = "https://finance.yahoo.com/quote/TSLA/financials?p=TSLA"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract revenue data from the financial table
    revenue_data = {}
    try:
        # Find the table with financial data
        table = soup.find('div', {'data-test': 'fin-row'})
        rows = table.find_all('div', {'class': 'D(tbr)'})

        for row in rows:
            cols = row.find_all('div', {'class': 'D(ib)'})
            if len(cols) > 1:
                period = cols[0].text.strip()
                revenue = cols[1].text.strip()
                revenue_data[period] = revenue
    except Exception as e:
        print(f"Error scraping revenue data: {e}")
    return revenue_data

# Fetch revenue data
revenue_data = get_tesla_revenue()

# Convert revenue data to a pandas DataFrame
revenue_df = pd.DataFrame(list(revenue_data.items()), columns=["Period", "Revenue"])

# Clean Revenue Data (remove 'B' for billions and convert to numerical values)
revenue_df["Revenue"] = revenue_df["Revenue"].str.replace("B", "").astype(float) * 1e9  # convert to full numbers

# Step 3: Create Stock Data Plot (Close Price)
stock_price_trace = go.Scatter(
    x=stock_data.index,
    y=stock_data['Close'],
    mode='lines',
    name='Tesla Stock Price',
    line=dict(color='blue')
)

# Step 4: Create Revenue Plot
revenue_trace = go.Bar(
    x=revenue_df["Period"],
    y=revenue_df["Revenue"],
    name='Tesla Revenue',
    marker=dict(color='orange')
)

# Step 5: Create Layout for the Dashboard
layout = go.Layout(
    title="Tesla Stock and Revenue Dashboard",
    xaxis=dict(title='Date/Period'),
    yaxis=dict(title='Amount (USD)'),
    barmode='group',
    template="plotly_dark"
)

# Step 6: Create Dash App
app = dash.Dash(__name__)

# App Layout
app.layout = html.Div(children=[
    html.H1("Tesla Stock and Revenue Dashboard", style={'text-align': 'center'}),

    # Stock Price Graph
    dcc.Graph(
        id='stock-price-graph',
        figure={
            'data': [stock_price_trace],
            'layout': layout
        }
    ),

    # Revenue Graph
    dcc.Graph(
        id='revenue-graph',
        figure={
            'data': [revenue_trace],
            'layout': layout
        }
    )
])

# Step 7: Run the Dash App
if __name__ == '__main__':
    app.run_server(debug=True)

Error scraping revenue data: 'NoneType' object has no attribute 'find_all'


<IPython.core.display.Javascript object>

Question 6 - GameStop Stock and Revenue Dashboard

In [38]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch GameStop Stock Data using yfinance
ticker = "GME"
stock_data = yf.download(ticker, start="2020-01-01", end="2024-01-01", progress=False)

# Step 2: Fetch GameStop Revenue Data from Yahoo Finance (Web Scraping)
def get_gamestop_revenue():
    url = "https://finance.yahoo.com/quote/GME/financials?p=GME"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    revenue_data = {}
    try:
        table = soup.find('div', {'data-test': 'fin-row'})
        rows = table.find_all('div', {'class': 'D(tbr)'})

        for row in rows:
            cols = row.find_all('div', {'class': 'D(ib)'})
            if len(cols) > 1:
                period = cols[0].text.strip()
                revenue = cols[1].text.strip()
                revenue_data[period] = revenue
    except Exception as e:
        print(f"Error scraping revenue data: {e}")
    return revenue_data

# Fetch revenue data
revenue_data = get_gamestop_revenue()

# Convert revenue data to pandas DataFrame
revenue_df = pd.DataFrame(list(revenue_data.items()), columns=["Period", "Revenue"])

# Clean Revenue Data (remove 'B' for billions and convert to numerical values)
revenue_df["Revenue"] = revenue_df["Revenue"].str.replace("B", "").astype(float) * 1e9  # convert to full numbers

# Step 3: Create Stock Data Plot (Close Price)
stock_price_trace = go.Scatter(
    x=stock_data.index,
    y=stock_data['Close'],
    mode='lines',
    name='GameStop Stock Price',
    line=dict(color='blue')
)

# Step 4: Create Revenue Plot
revenue_trace = go.Bar(
    x=revenue_df["Period"],
    y=revenue_df["Revenue"],
    name='GameStop Revenue',
    marker=dict(color='orange')
)

# Step 5: Create Layout for the Dashboard
layout = go.Layout(
    title="GameStop Stock and Revenue Dashboard",
    xaxis=dict(title='Date/Period'),
    yaxis=dict(title='Amount (USD)'),
    barmode='group',
    template="plotly_dark"
)

# Step 6: Create Dash App
app = dash.Dash(__name__)

# App Layout
app.layout = html.Div(children=[
    html.H1("GameStop Stock and Revenue Dashboard", style={'text-align': 'center'}),

    # Stock Price Graph
    dcc.Graph(
        id='stock-price-graph',
        figure={
            'data': [stock_price_trace],
            'layout': layout
        }
    ),

    # Revenue Graph
    dcc.Graph(
        id='revenue-graph',
        figure={
            'data': [revenue_trace],
            'layout': layout
        }
    )
])

# Step 7: Run the Dash App
if __name__ == '__main__':
    app.run_server(debug=True)

Error scraping revenue data: 'NoneType' object has no attribute 'find_all'


<IPython.core.display.Javascript object>